# Saving the state of the random number generator
Credits: http://www.cookbook-r.com/ (Creative Commons Attribution-Share Alike 3.0 Unported License)

## Problem
You want to save and restore the state of the random number generator

## Solution
Save .Random.seed to another variable and then later restore that value to .Random.seed.

In [1]:
# For this example, set the random seed
set.seed(423)
runif(3)

# Save the seed
oldseed <- .Random.seed

[1] 0.1089715 0.5973455 0.9726307

In [2]:
runif(3)

# Do some other stuff with RNG here, such as:
runif(30)
# ...

[1] 0.7973768 0.2278427 0.5189830

[1] 0.69292553 0.81044534 0.10194650 0.91666524 0.20249243 0.16527665
 [7] 0.46399420 0.13005193 0.89368600 0.63863225 0.19254541 0.57557113
[13] 0.78458928 0.55080956 0.59388856 0.02040073 0.17663162 0.33647923
[19] 0.53304330 0.22939499 0.43232959 0.71889969 0.97318020 0.20320008
[25] 0.58991593 0.88450876 0.61154896 0.68123761 0.48162899 0.51807563

In [3]:
# Restore the seed
.Random.seed <- oldseed

# Get the same random numbers as before, after saving the seed
runif(3)
#> [1] 0.7973768 0.2278427 0.5189830

[1] 0.7973768 0.2278427 0.5189830

If no random number generator has been used in your R session, the variable .Random.seed will not exist. If you cannot be certain that an RNG has been used before attempting to save, the seed, you should check for it before saving and restoring:

In [4]:
oldseed <- NULL
if (exists(".Random.seed"))
    oldseed <- .Random.seed

# Do some other stuff with RNG here, such as:
runif(30)

if (!is.null(oldseed))
    .Random.seed <- oldseed

[1] 0.69292553 0.81044534 0.10194650 0.91666524 0.20249243 0.16527665
 [7] 0.46399420 0.13005193 0.89368600 0.63863225 0.19254541 0.57557113
[13] 0.78458928 0.55080956 0.59388856 0.02040073 0.17663162 0.33647923
[19] 0.53304330 0.22939499 0.43232959 0.71889969 0.97318020 0.20320008
[25] 0.58991593 0.88450876 0.61154896 0.68123761 0.48162899 0.51807563

## Saving and restoring the state of the RNG in functions
If you attempt to restore the state of the random number generator within a function by using ```.Random.seed <- x```, it will not work, because this operation changes a _local_ variable named `.Random.seed`, instead of the variable in the _global_ envrionment.

Here are two examples. What these functions are supposed to do is generate some random numbers, while leaving the state of the RNG unchanged.

In [5]:
# This is the bad version
bad_rand_restore <- function() {
    if (exists(".Random.seed"))
        oldseed <- .Random.seed
    else
        oldseed <- NULL

    print(runif(3))

    if (!is.null(oldseed))
        .Random.seed <- oldseed
    else
        rm(".Random.seed")
}

In [6]:
# This is the good version
rand_restore <- function() {
    if (exists(".Random.seed", .GlobalEnv))
        oldseed <- .GlobalEnv$.Random.seed
    else
        oldseed <- NULL

    print(runif(3))

    if (!is.null(oldseed)) 
        .GlobalEnv$.Random.seed <- oldseed
    else
        rm(".Random.seed", envir = .GlobalEnv)
}

In [7]:
# The bad version doesn't properly reset the RNG state, so random numbers keep
# changing.
set.seed(423)
bad_rand_restore()
bad_rand_restore()
bad_rand_restore()

[1] 0.1089715 0.5973455 0.9726307
[1] 0.7973768 0.2278427 0.5189830
[1] 0.6929255 0.8104453 0.1019465


In [8]:
# The good version correctly resets the RNG state each time, so random numbers
# stay the same.
set.seed(423)
rand_restore()
rand_restore()
rand_restore()

[1] 0.1089715 0.5973455 0.9726307
[1] 0.1089715 0.5973455 0.9726307
[1] 0.1089715 0.5973455 0.9726307


## Notes

.Random.seed should not be modified by the user.